# Практическое занятие №5

П.Н. Советов, РТУ МИРЭА

Решать эту и дальнейшие задачи удобнее в парном составе: один студент занимается написанием кода программы и пытается внести в код такие ошибки, которые будет сложно протестировать другому студенту.

Пример программы с ошибкой:

```Python
def bucketsort(arr, k):
    counts = [0] * k
    for x in arr:
        counts[x] += 1

    sorted_arr = []
    for i, count in enumerate(arr):
        sorted_arr.extend([i] * count)

    return sorted_arr
```

Другие примеры можно взять [отсюда](https://github.com/jkoppel/QuixBugs/tree/master/python_programs).

**Задача 1**

Использование модуля doctest

1. Добавьте документацию к программе в виде docstring-строки.
1. Укажите примеры в формате doctest. Примеры должны охватывать граничные случаи.
1. Протестируйте программу с помощью вызова модуля doctest.
1. Перенесите примеры в отдельный файл и снова протестируйте программу.

In [9]:
import doctest

def factorial(n):
    """
    >>> [factorial(n) for n in range(6)]
    [1, 1, 2, 6, 24, 120]
    
    >>> factorial(30)
    265252859812191058636308480000000
    >>> factorial(-1)
    Traceback (most recent call last):
        ...
    ValueError: n must be >= 0

    Factorials of floats are OK, but the float must be an exact integer:
    >>> factorial(30.1)
    Traceback (most recent call last):
        ...
    ValueError: n must be exact integer
    >>> factorial(30.0)
    265252859812191058636308480000000

    It must also not be ridiculously large:
    >>> factorial(1e100)
    Traceback (most recent call last):
        ...
    OverflowError: n too large"""
    import math
    if not n >= 0:
        raise ValueError("n must be >= 0")
    if math.floor(n) != n:
        raise ValueError("n must be exact integer")
    if n+1 == n:  # catch a value like 1e300
        raise OverflowError("n too large")
    result = 1
    factor = 2
    while factor <= n:
        result *= factor
        factor += 1
    return result

if __name__ == "__main__":

    doctest.testmod()

**Задача 2**

Использование модуля pytest

1. Создайте отдельный файл для тестирования в который поместите тестирующие функции (не менее двух).
1. Упростите код с помощью добавления fixture-функций.
1. Добавьте параметризацию.
1. Добавьте макетный код.

**Задача 3**

Использование модуля coverage

1. Получите статистику по покрытию операторов.
1. Получите статистику по покрытию ветвей.
1. Постарайтесь изменить код исходной программы так, чтобы затруднить получение 100% покрытия.
1. Реализуйте вывод статистики о покрытии в HTML-представлении.

In [10]:
import math


def func1(x):
    if (x < 15):
        return("x < 15")
    elif (x > 15):
        return("x > 15")
    else:
        return("x = 15")

def func2(x):
    for i in range(x**5):
        x += x
    return(x)

def func_cos(x, n):
    cos_approx = 0
    for i in range(n):
        coef = (-1)**i
        num = x**(2*i)
        denom = math.factorial(2*i)
        cos_approx += ( coef ) * ( (num)/(denom) )
    
    return cos_approx

def test_func1():
    assert func1(5) == "x < 15"
    assert func1(11) == "x < 15"

def test_func2():
    assert func2(1) == 2
    assert func2(2) == 131072

def test_func_cos():
    assert func_cos(5, 20) == 0.28366218546322675
    assert func_cos(11, 30) == 0.004425697988363761
    assert func_cos(6, 6) == -2.8057142857142807


**Задача 4**

Прототип системы мутационного тестирования приведен ниже. Попробуйте разобраться в том, как работает этот код. Вам поможет документация к модулю ast.

Функция `mut_test` принимает на вход тестируемую функцию и функцию, осуществляющую тестирование с помощью `assert`. Функции необходимо добавлять к приведенному ниже модулю.

In [11]:
import random
from collections import defaultdict
import inspect
from ast import *
import math


class Locator(NodeVisitor):
    def __init__(self):
        self.locs = defaultdict(list)

    def visit(self, node):
        self.locs[type(node)].append(node)
        self.generic_visit(node)


class Mutator(NodeTransformer):
    def __init__(self, locs):
        node_type = random.choice([BinOp, Constant])
        self.target = random.choice(locs[node_type])
        i = 0

    def visit(self, node):
        if self.target != node:
            return self.generic_visit(node)
        elif isinstance(node, Constant):
            node2 = Constant()
            node2.value = random.choice([1, 3, 4, 5])
            node2.kind = node.kind
            copy_location(node2, node)
            return self.generic_visit(node2)
        elif isinstance(node, BinOp):
            node2 = node
            node2.op = random.choice([Add(), Sub(), Div(), Mult(), Mod(), Pow()])
            return self.generic_visit(node2)


def mutate_code(src, max_changes):
    tree = parse(src)
    loc = Locator()
    loc.visit(tree)
    mut = Mutator(loc.locs)
    for _ in range(random.randint(1, max_changes)):
        mut.visit(tree)
    return unparse(tree)


def make_mutants(func, size, max_changes):
    mutant = src = unparse(parse(inspect.getsource(func)))
    mutants = [src]
    while len(mutants) < size + 1:
        while mutant in mutants:
            mutant = mutate_code(src, max_changes)
        mutants.append(mutant)

    return mutants[1:]


def mut_test(func, test, size=20, max_changes=1):
    survived = []
    mutants = make_mutants(func, size, max_changes)
    for mutant in mutants:
        try:
            exec(mutant, globals())
            test()
            survived.append(mutant)
        except:
            pass
    return survived


def dist(x1, y1, x2, y2):
    return math.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)

def test_dist():
    assert dist(4, 4, 4, 4) == 0
    

mut_test(dist, test_dist)

['def dist(x1, y1, x2, y2):\n    return math.sqrt((x1 - x2) ** 5 + (y1 - y2) ** 2)',
 'def dist(x1, y1, x2, y2):\n    return math.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 3)',
 'def dist(x1, y1, x2, y2):\n    return math.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 1)',
 'def dist(x1, y1, x2, y2):\n    return math.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 4)',
 'def dist(x1, y1, x2, y2):\n    return math.sqrt((x1 - x2) ** 2 + (y1 - y2) * 2)',
 'def dist(x1, y1, x2, y2):\n    return math.sqrt((x1 - x2) ** 4 + (y1 - y2) ** 2)',
 'def dist(x1, y1, x2, y2):\n    return math.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 5)',
 'def dist(x1, y1, x2, y2):\n    return math.sqrt((x1 - x2) ** 3 + (y1 - y2) ** 2)',
 'def dist(x1, y1, x2, y2):\n    return math.sqrt((x1 - x2) ** 1 + (y1 - y2) ** 2)',
 'def dist(x1, y1, x2, y2):\n    return math.sqrt((x1 - x2) ** 2 + (y1 % y2) ** 2)',
 'def dist(x1, y1, x2, y2):\n    return math.sqrt((x1 - x2) ** 2 + (y1 - y2) / 2)',
 'def dist(x1, y1, x2, y2):\n    return math.sqrt((x1 - x2) ** 2 * 

In [4]:
def dist(x1, y1, x2, y2):
    return math.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)

def test_dist():
    assert dist(1, 2, 3, 4) == 2
    assert dist(4, 4, 4, 4) == 0
    

mut_test(dist, test_dist)

['def dist(x1, y1, x2, y2):\n    return math.sqrt((x1 - x2) % 2 + (y1 - y2) ** 2)']

1. Сгенерируйте несколько программ-мутантов.
1. Сгенерируйте несколько программ-мутантов при 100% покрытии ветвей.
1. Постарайтесь добиться полного отсутствия мутантов.
1. Было бы удобно общий код мутационного тестирования вынести в отдельный модуль. Увы, это не сработает. Почему?

**Задача 5**

Использование модуля deal

1. Добавьте к программе контракты `pre`, `post`, `ensure`, `raises`, `reason`.
1. Найдите программу, для которой контракт `has` будет полезен.
1. Для программы с классами используйте инвариант `inv`.
1. Для тестирования контрактов используйте pytest.
1. Реализуйте контракт, выполнение которого `deal` проверяет статически. Какие ограничения имеют статически проверяемые контракты?

In [12]:
import math

import deal
import pytest
import random

class TooManyIterationsException(Exception):
    """Raised when too many iterations"""
    pass

@deal.pre(lambda x, n: n > 0)
@deal.post(lambda cos_approx: -1 <= cos_approx <= 1)
@deal.ensure(lambda x, n, result: round(result, 1) == round(math.cos(x), 1))
@deal.reason(TooManyIterationsException, lambda x, n: n > 100)
@deal.raises(TooManyIterationsException)
@deal.has('stdout')

def func_cos(x, n):
    if (n > 100):
        raise TooManyIterationsException
    cos_approx = 0
    for i in range(n):
        coef = (-1)**i
        num = x**(2*i)
        denom = math.factorial(2*i)
        cos_approx += ( coef ) * ( (num)/(denom) )
    
    print(cos_approx)
    return cos_approx


func_cos(5, 100)

@deal.inv(lambda game: game.level >= 1)
class Game:
    def __init__(self):
        self.level = 1

    def passLevel(self):
        self.level += 1

    def goBack(self, levelNum):
        if (levelNum < self.level):
            self.level = levelNum
        else:
            print("You can not go to level you didn't pass")


g = Game()

g.passLevel()
g.goBack(-1)

print(g.level)

@pytest.fixture
def new_game():
    g = Game()
    return g

def test_game1(new_game):
    for i in range(10):
        new_game.passLevel()
    new_game.goBack(4)
    assert new_game.level == 4

def test_game2(new_game):
    with pytest.raises(deal.InvContractError):
        new_game.goBack(-3)

0.28366218546322675


InvContractError: expected game.level >= [34m1[39;49;00m

**Задача 6**

Использование модуля hypothesis

1. Реализуйте тестирование для численных входных данных, строк, списков.
1. Реализуйте тестирование для словарей, деревьев, графов.
1. Используйте при создании свойств как можно больше категорий, перечисленных в лекции.
1. Найдите подходящую программу и реализуйте для нее тестирование на основе модели.
1. Разберитесь, для чего в библиотеке hypothesis используются bundles и примените их в тестировании на основе моделей.

In [13]:
import shutil
import tempfile
import unittest
from collections import defaultdict

from hypothesis.strategies import composite
from hypothesis import given
from hypothesis import strategies as hps
from hypothesis import strategies as st
from hypothesis.database import DirectoryBasedExampleDatabase
from hypothesis.stateful import (Bundle, RuleBasedStateMachine, precondition,
                                 rule)
from hypothesis.strategies import dictionaries, integers, text


# String
def mode(data):
    return max(data, key=data.count)

@given(data=st.lists(st.integers(), min_size=1))
def test_mode(data):
    res = mode(data)
    assert res in data
    assert all(data.count(res) >= data.count(x) for x in data)
test_mode()

def encode(input_string):
    count = 1
    prev = ""
    lst = []
    for character in input_string:
        if character != prev:
            if prev:
                entry = (prev, count)
                lst.append(entry)
            count = 1
            prev = character
        else:
            count += 1
        entry = (character, count)
        lst.append(entry)
    return lst

def decode(lst):
    q = ""
    for character, count in lst:
        q += character * count
    return q

@given(text())
def test_decode_inverts_encode(s):
    assert decode(encode(s)) == s

class TestEncoding(unittest.TestCase):
    @given(text())
    def test_decode_inverts_encode(self, s):
        self.assertEqual(decode(encode(s)), s)


# Integer example

def add(x, y):
    return x + y

@given(x=st.integers(), y=st.integers(), z=st.integers())
def test_add(x, y, z):
    assert add(x, y) == add(y, x)
    assert add(x, add(y, z)) == add(add(x, y), z)
    
test_add()


# List example

def mode(data):
    return max(data, key=data.count)

@given(data=st.lists(st.integers()))
def test_mode(data):
    assert mode(data) in data


#Tree 
class TreeNode:
    def __init__(self, x):
        self.val = x
        self.left = None
        self.right = None

    def __repr__(self):
        if ((not self.left) and (not self.right)):
            return f'TreeNode({self.val})'
        return f'TreeNode({self.val}, left={self.left}, right={self.right}'


def is_valid_bst(node):
    if (not node):
        return True

    is_valid = True
    if (node.left):
        is_valid = is_valid and node.val > node.left.val
    if (node.right):
        is_valid = is_valid and node.val < node.right.val

    if (not is_valid):
        return False

    return is_valid_bst(node.left) and is_valid_bst(node.right)

@composite
def valid_bst_trees(
    draw, tree_strategy, min_value=None, max_value=None):
    node = draw(tree_strategy)
    if (not node):
        return None
    
    if ((min_value is not None) and (max_value is not None) and (min_value >= max_value)):
        return None

    val = draw(integers(min_value=min_value, max_value=max_value))
    node.val = val

    node.left = draw(valid_bst_trees(
        tree_strategy=tree_strategy,
        min_value=min_value,
        max_value=node.val - 1))
    node.right = draw(valid_bst_trees( tree_strategy=tree_strategy, min_value=node.val + 1, max_value=max_value))
    
    return node


def gen_bst(tree_strategy, min_value=None, max_value=None):
  return valid_bst_trees(
      tree_strategy=tree_strategy,
      min_value=min_value,
      max_value=max_value)

singleton_tree = hps.just(-111).map(TreeNode)

@given(hps.recursive(hps.just(None) | singleton_tree, gen_bst))
def test_is_valid_bst_works(node):
    assert is_valid_bst(node)

def test_is_valid_bst():
    assert is_valid_bst(None)
    assert is_valid_bst(TreeNode(1))

    node1 = TreeNode(1)
    node1.left = TreeNode(0)
    assert is_valid_bst(node1)

    node2 = TreeNode(1)
    node2.left = TreeNode(1)
    assert not is_valid_bst(node2)

    node3 = TreeNode(1)
    node3.left = TreeNode(0)
    node3.right = TreeNode(1)
    assert not is_valid_bst(node3)

@given(st.builds(zip, st.lists(st.text(min_size=1, max_size=10), min_size=2, unique=True),
                st.lists(st.from_type(type).flatmap(st.from_type).filter(lambda x: not isinstance(x, (type(None)))),
                min_size=2,
                unique_by=lambda x: type(x),
)).map(dict))

def test_dict(dictionary):
    values_types = list(map(type, dictionary.values()))
    assert len(set(values_types)) == len(values_types)

class Stack:
    def __init__(self, size):
        self.data = [0] * size
        self.sp = 0
   
    def push(self, x):
        self.data[self.sp] = x
        self.sp += 1

    def get_top(self):
        return self.data[self.sp - 1]

    def dup(self):
        self.push(self.get_top())
        
    def pop(self):
        x = self.get_top()
        self.sp -= 1
        return x
    
    def swap(self):
        y, x = self.pop(), self.pop()
        self.push(x)
        self.push(y)


class ModelStack(RuleBasedStateMachine):
    def __init__(self):
        super().__init__()
        self.data = []
        self.stack = Stack(100)

    @rule(x=st.integers())
    def push(self, x):
        self.stack.push(x)
        self.data.append(x)

    @rule()
    @precondition(lambda self: len(self.data))
    def dup(self):
        self.stack.dup()
        self.data.append(self.data[-1])

    @rule()
    @precondition(lambda self: len(self.data))
    def pop(self):
        self.stack.pop()        
        return self.data.pop()
          
    @rule()
    @precondition(lambda self: len(self.data) > 1)
    def swap(self):
        self.stack.swap()
        self.data[-1], self.data[-2] = self.data[-2], self.data[-1]

    @rule()
    def check_stacks(self):
        assert self.stack.data[:self.stack.sp] == self.data


test_stack = ModelStack.TestCase

class DatabaseComparison(RuleBasedStateMachine):
    def __init__(self):
        super().__init__()
        self.tempd = tempfile.mkdtemp()
        self.database = DirectoryBasedExampleDatabase(self.tempd)
        self.model = defaultdict(set)

    keys = Bundle("keys")
    values = Bundle("values")

    @rule(target=keys, k=st.binary())
    def add_key(self, k):
        return k

    @rule(target=values, v=st.binary())
    def add_value(self, v):
        return v

    @rule(k=keys, v=values)
    def save(self, k, v):
        self.model[k].add(v)
        self.database.save(k, v)

    @rule(k=keys, v=values)
    def delete(self, k, v):
        self.model[k].discard(v)
        self.database.delete(k, v)

    @rule(k=keys)
    def values_agree(self, k):
        assert set(self.database.fetch(k)) == self.model[k]

    def teardown(self):
        shutil.rmtree(self.tempd)

TestDBComparison = DatabaseComparison.TestCase


**Задача 7**

Формальная верификация головоломок из компьютерных игр

Одной из важных проблем, стоящих перед разработчиком компьютерных игр, является создание интересных головоломок, в которых отсутствуют тупиковые состояния (состояния, из которых нельзя достичь цели).

Формальную верификацию проведем следующим образом:

1. Реализовать игровую ситуацию в виде некоторого количества локаций с указанием перечня возможных действий в каждой из них.
1. Сгенерировать по реализованной игровой ситуации граф всех возможных игровых состояний, в котором ребра задают переходы из состояние в состояние.
1. Проанализировать граф состояний на предмет проверяемого игрового свойства.

Рассмотрим следующую игровую ситуацию из [PuzzleGraph](https://runevision.itch.io/puzzlegraph):

![](images/puzzlegraph.png)

Здесь Goal означает целевое состояние, а состояние с двумя зелеными кругами обозначает старт. Справа от стартового состояния используется специальное ребро, по которому позволяется двигаться только в одну сторону, вправо.

Ниже приведено описание рассмотренной игровой ситуации на Питоне:

In [14]:
import graphviz

# Функция перехода из комнаты в комнату
def go(room):
    def func(state):
        return {state: room}
    return func


# Структура игры. Комнаты и допустимые в них действия
game = {
    'room0': dict(
        left=go('room1'),
        up=go('room2'),
        right=go('room3')
    ),
    'room1': dict(
        up=go('room2'),
        right=go('room0')
    ),
    'room2': dict(
    ),
    'room3': dict(
        up=go('room4'),
        right=go('room5')
    ),
    'room4': dict(
        down=go('room3'),
        right=go('room5')
    ),
    'room5': dict(
        up=go('room4'),
        left=go('room3')
    )
}

def switch():
    game["room3"]["left"] = go('room0')


START_STATE = dict(room='room0')
END_ROOM = "room2"


def pathExists(graph, start, goal):
    explored = []
    queue = [[start]]
     
    if start == goal:
        return True

    while queue:
        path = queue.pop(0)
        node = path[-1]

        if node not in explored:
            neighbours = graph[node]

            for neighbour in neighbours:
                new_path = list(path)
                new_path.append(neighbour)
                queue.append(new_path)

                if neighbour == goal:
                    print("Shortest path = ", *new_path)
                    return True
            explored.append(node)
 
    return False

def get_model(currState, parent, passedRooms, graph):
    currRoom = currState[parent]
    graph[currRoom] = []
    passedRooms.append(currRoom)
    for dir in game[currState[parent]]:
        nextRoom = (game[currRoom][dir](currRoom))[currRoom]
        graph[currRoom].append(nextRoom)
        if (nextRoom in passedRooms):
            continue
        get_model({currRoom: nextRoom}, currRoom, passedRooms, graph)

def makeGraph(graph, end):
    dot = graphviz.Digraph(comment='Ways', strict=True)

    dot.node(START_STATE['room'], shape="circle", style='filled', fillcolor='blue')

    for key in graph.keys():
        if (not pathExists(graph, key, END_ROOM)):
            dot.node(key, shape="circle", style='filled', fillcolor='red')
        else:
            dot.node(key, shape="circle")
        for value in graph[key]:
            dot.node(value, shape="circle")
            dot.edge(key, value)

    dot.node(end, shape="circle", style='filled', fillcolor='green')

    return dot

#switch()
graph = {}
get_model(START_STATE, "room", [], graph)

dot = makeGraph(graph, END_ROOM)

g = dot.render(filename="graph.gv")
s = graphviz.Source.from_file("graph.gv")
s.view()

Shortest path =  room0 room2
Shortest path =  room1 room2


'graph.gv.pdf'

1. Реализовать функцию `make_model`, которая по структуре игры и стартовому состоянию строит граф всех возможных состояний.
1. Реализовать функцию `find_dead_ends`, которая выдает список тупиковых узлов графа. Вспомните тупиковые ситуации из известных вам компьютерных игр, где, иной раз, для дальнейшего прохождения нужно прибегать к старому сохранению или вводу специальных системных команд, и все потому, что был потерян важный для дальнейшего прохождения предмет или не совершено требуемое действие.
1. Добавьте в одну из комнат room3-room5 рычаг, нажатие на который делает односторонний переход из room0 в room3 двусторонним.
1. (дополнительно) Придумайте минимальную игровую ситуацию, при которой будет наблюдаться комбинаторный взрыв для графа состояний.

Простой вариант функции печати графа в формате GraphViz:

In [8]:
def print_dot(graph, start_key):
    dead_ends = [] # find_dead_ends(graph) TODO
    print('digraph {')
    graph_keys = list(graph.keys())
    for x in graph:
        n = graph_keys.index(x)
        if x == start_key:
            print(f'n{n} [style="filled",fillcolor="dodgerblue",shape="circle"]')
        elif is_goal_state(x):
            print(f'n{n} [style="filled",fillcolor="green",shape="circle"]')
        elif x in dead_ends:
            print(f'n{n} [style="filled",fillcolor="red",shape="circle"]')
        else:
            print(f'n{n} [shape="circle"]')
    for x in graph:
        n1 = graph_keys.index(x)
        for y in graph[x]:
            n2 = graph_keys.index(y)
            print(f'n{n1} -> n{n2}')
    print('}')

Вот как выглядит вывод функции print_dot:
    
![](images/game.svg)

Вывод для пункта 4 с рычагом:
    
![](images/puzzlegraph2.svg)

**Задача 8**

Есть простая игра The Teeny Tiny Mansion (TTTM), описание которой приведено по [ссылке](http://svn.clairexen.net/handicraft/2017/tttm/README). В этой игре имеется 2 персонажа (Алиса и Боб), которыми можно поочередно управлять, 4 комнаты, 3 двери и 3 ключа. Персонажи могут брать ключи и передавать их друг другу. Целью является привести Алису в красную комнату, а Боба — в голубую комнату.

В качестве стартового состояния используйте:

```Python

START_STATE = dict(
    player='alice',
    alice_room='west room',
    bob_room='east room',
    red_key='east room',
    blue_key='west room',
    green_key='east room'
)
```

1. Реализовать все пункты предыдущей задачи для TTTM.
1. Реализовать функцию подсчета кратчайшего числа шагов, за которые игру можно успешно завершить.
1. Провести верификацию для общего случая: Алиса, Боб и 3 ключа случайно располагаются в западной и восточной комнатах.

Пример вывода графа состояний:

![](images/tttm.svg)